In [1]:
%pwd

u'C:\\Users\\rek\\Desktop\\GH\\cabi'

In [2]:
import cabi_Func as cf
import os
import pickle
import dill
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
from sklearn import metrics
from sklearn import grid_search

In [3]:
dbName = 'cabi_000'
tableTH = 'TH'
tableW  = 'Weather'
tableS  = 'Stations'
tableD = 'docksMerged'
tableB = 'bikesMerged'
weatherFile = 'KDCA___csv_mesowest_2010-2015.csv'
stationsCSV = 'stationInfo_v8.csv'

In [4]:
fB = open('save_bikes0912_000.p','rb')
fD = open('save_docks0912_000.p','rb')
bDF = pickle.load(fB)
dDF = pickle.load(fD)
S = cf.rek_readSQL(dbName,tableS)
S.drop('index', axis=1, inplace=True)
S.index = S.terminalname
stationStrings = map(str,S.terminalname)

In [5]:
predictorColumnNames = [cc for cc in dDF.columns if ('D_' not in cc)]
responseColumnsNamesB = [cc for cc in bDF.columns if ('B_' in cc)]
responseColumnsNamesD = [cc for cc in dDF.columns if ('D_' in cc)]
RCN = responseColumnsNamesB+responseColumnsNamesD

fullDF = pd.concat([dDF[predictorColumnNames],
                                bDF[responseColumnsNamesB],
                                dDF[responseColumnsNamesD]],axis=1)
allDemand = pd.DataFrame()
for ss in stationStrings:
    for bd_ in ['B_','D_']:
        allDemand[bd_+ss] = fullDF[[x for x in RCN if ((bd_ in x) and (ss in x))]].sum(axis=1)
systemWideDemand = pd.DataFrame()
for bd_ in ['B_','D_']:
    systemWideDemand[bd_] = allDemand[[x for x in allDemand.columns if x.startswith(bd_)]].sum(axis=1)
predictors = dDF[predictorColumnNames].drop(['timeW','precip06h','dewpointF'],1)

In [6]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']
from matplotlib import pylab as plt

In [ ]:
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(predictors, allDemand, test_size=0.2, random_state=42)

In [7]:
estRF = RandomForestRegressor(max_features=3,min_samples_leaf=50,n_estimators=30,n_jobs=4)

In [8]:
#%time myFit = estRF.fit(X_train, y_train)
%time myFit = estRF.fit(predictors,allDemand)

Wall time: 5.53 s


In [9]:
%time pp = myFit.predict(predictors.iloc[0:1,:])

Wall time: 127 ms


In [10]:
predictors.iloc[400:401,:]

,DOW,DOY,hour,isHol,year,tempF,RH,windSpeed,precip01h,snowDepth
383900,4,290,16,0,2013,73.0,71.0,9.2,0.0,0.0


In [11]:
myFit.predict(predictors.iloc[400:401,:])

array([[  3.22409634e-01,   2.02221138e-01,   3.82076299e-01,
          3.72566658e-01,   1.09560199e+00,   1.23283152e+00,
          9.03070127e-01,   8.90631874e-01,   4.42915533e-01,
          5.12521432e-01,   1.19881319e+00,   1.23734213e+00,
          6.89686659e-01,   7.26583441e-01,   2.36536272e+00,
          1.93660453e+00,   6.41826801e-01,   7.30299806e-01,
          1.35036451e+00,   9.30197324e-01,   6.36749449e-01,
          1.05450279e+00,   1.25056899e+00,   1.12098183e+00,
          5.46887812e-01,   4.30200062e-01,   3.36721383e-01,
          4.86070513e-01,   2.76429220e+00,   3.22876379e+00,
          1.73945628e+00,   2.08717424e+00,   9.23764755e-01,
          8.68357491e-01,   2.08999957e+00,   1.15335812e+00,
          5.58999355e-01,   4.30415135e-01,   6.73218456e-01,
          6.27119610e-01,   9.30783014e-01,   7.98958927e-01,
          5.86464655e-01,   4.83790913e-01,   1.26106384e+00,
          9.52115830e-01,   6.15246947e-01,   6.34852796e-01,
        

In [12]:
dill.dump(myFit,open("rafo_30.p","wb"))
#dill.dump(S,open("S_000.p","wb"))

In [11]:
S.head()

,terminalname,name,lat,long,el,juris,neigh,firstUse,lastUse
terminalname,,,,,,,,,
31000,31000,20th & Bell St,38.856100,-77.051200,50.8,Arlington,Crystal City,9/20/2010,12/31/2015
31001,31001,18th & Eads St.,38.857250,-77.053320,45.2,Arlington,Crystal City,4/1/2013,12/31/2015
31002,31002,20th & Crystal Dr,38.856214,-77.049828,39.3,Arlington,Crystal City,9/20/2010,12/31/2015
31003,31003,15th & Crystal Dr,38.860170,-77.049593,38.4,Arlington,Crystal City,9/20/2010,12/31/2015
31004,31004,Aurora Hills Community Ctr/18th & Hayes St,38.857866,-77.059490,53.1,Arlington,Pentagon City,9/20/2010,12/31/2015


In [12]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [15]:
gjS = df_to_geojson(S,['terminalname','name'],lat='lat',lon='long')

In [23]:
(gjS['features'])[:2]

[{'geometry': {'coordinates': [-77.0512, 38.8561], 'type': 'Point'},
  'properties': {'name': u'20th & Bell St', 'terminalname': 31000L},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-77.05332, 38.85725], 'type': 'Point'},
  'properties': {'name': u'18th & Eads St.', 'terminalname': 31001L},
  'type': 'Feature'}]

In [25]:
gjS['features']

[{'geometry': {'coordinates': [-77.0512, 38.8561], 'type': 'Point'},
  'properties': {'name': u'20th & Bell St', 'terminalname': 31000L},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-77.05332, 38.85725], 'type': 'Point'},
  'properties': {'name': u'18th & Eads St.', 'terminalname': 31001L},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-77.049828, 38.856214], 'type': 'Point'},
  'properties': {'name': u'20th & Crystal Dr', 'terminalname': 31002L},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-77.049593, 38.860170000000004],
   'type': 'Point'},
  'properties': {'name': u'15th & Crystal Dr', 'terminalname': 31003L},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-77.05949, 38.857865999999994],
   'type': 'Point'},
  'properties': {'name': u'Aurora Hills Community Ctr/18th & Hayes St',
   'terminalname': 31004L},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-77.059936, 38.862303000000004],
   'type': 'Point'},
  'properties': {'name': u'Pentagon City

In [ ]:
print(predictors.shape)
print(allDemand.shape)

In [ ]:
plt.plot(pp, y_test, 'b.', alpha=0.1)
plt.xlabel('predicted')
plt.ylabel('observed')
plt.show()

In [36]:
#print(pp.shape)
pp[0,0]
#print(y_test.shape)
#S.head()

0.023017185551413775

In [33]:
S['terminalname'].iloc[3]

31003

In [ ]:
min(pp.sum(axis=0))

In [ ]:
S.loc[int(y_test.columns[200][2:])]

In [ ]:
metrics.r2_score(y_test.iloc[:,200], pp[:,200])

In [ ]:
fullcount_716 = allDemand.sum(axis=0)
r2_716=[]
for x in range(716):
    r2_716.append(metrics.r2_score(y_test.iloc[:,x], pp[:,x]))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(fullcount_716, r2_716, 'b.', alpha=0.3)
ax.set_xscale('log')
ax.set_xlabel('Total number of rides')
ax.set_ylabel('R2 of random forest regression fit')

In [ ]:
allDemand.shape

In [ ]:
allDemand.sum(axis=0).shape